<a href="https://colab.research.google.com/github/Abhishek-Prajapat/Learning-Ai-from-scratch/blob/master/Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
import json
token = {"username":"abhishekprajapat","key":"f8c7968fc7514f21e97a0e9a099be45e"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [4]:
try:
  !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
except:
  try:
    !kaggle config set -n path 
  except:
    !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

finally:
  !kaggle config set -n path 

usage: kaggle config set [-h] -n NAME -v VALUE
kaggle config set: error: argument -v/--value is required


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
!kaggle datasets list

ref                                                           title                                                size  lastUpdated          downloadCount  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
allen-institute-for-ai/CORD-19-research-challenge             COVID-19 Open Research Dataset Challenge (CORD-19)    3GB  2020-06-03 23:32:41          71785  
roche-data-science-coalition/uncover                          UNCOVER COVID-19 Challenge                          179MB  2020-05-21 18:57:53          10483  
stefanlarson/outofscope-intent-classification-dataset         Out-of-Scope Intent Classification Dataset          285KB  2020-05-15 03:04:34              1  
ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney      TV shows on Netflix, Prime Video, Hulu and Disney+   88KB  2020-05-25 15:38:39             12  
gomes555/road-transport-brazil                      

In [7]:
! kaggle datasets download -d snap/amazon-fine-food-reviews

amazon-fine-food-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt

In [0]:
con = sqlite3.connect('./database.sqlite')

In [0]:
raw_data = pd.read_sql_query('''
SELECT * FROM Reviews
''', con)

In [12]:
raw_data.shape

(568454, 10)

In [0]:
data = pd.read_sql_query('''
SELECT * FROM Reviews
WHERE Score != 3''', con)

In [14]:
data.shape

(525814, 10)

In [0]:
def pos_neg(value):
  if value < 3:
    return 'negative'
  elif value > 3:
    return 'positive'
  
actual_score = data['Score']
changed_score = actual_score.map(pos_neg)
data['Score'] = changed_score

In [16]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [0]:
sorted_data = data.sort_values(by = ['ProductId'])

In [0]:
final_data = sorted_data.drop_duplicates(subset = {'UserId', 'ProfileName','Time','Text'})

In [19]:
final_data.shape, sorted_data.shape

((364173, 10), (525814, 10))

In [0]:
final_data = final_data[final_data['HelpfulnessDenominator'] >= final_data['HelpfulnessNumerator']]

In [21]:
final_data.shape

(364171, 10)

# Text Preprocessing:
1.   clean html, punc etc.
2.   lower case
3.   stopwords
4.   stemming


In [0]:
import re
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

In [23]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [0]:
def cleanhtml(sentence):
  cleanr = re.compile('<.*?>')
  cleaned_sentence = re.sub(cleanr, ' ', sentence)
  return cleaned_sentence

def cleanpunc(sentence):
  cleaned = re.sub(r'[#|:|@|$|!|?|,|\'|"|;|/|\|^|(|)|.]', ' ', sentence)
  return cleaned


In [0]:
sno = SnowballStemmer('english')

In [28]:
filtered_sentence = []
i = 0
positive_words = []
negative_words = []

for sent in final_data['Text'].values:
  sqrl = ''
  filtered_words = []
  sent = cleanhtml(sent)
  sent = cleanpunc(sent)

  for w in sent.split():
    w = w.lower()
    if (len(w)>2) and (w not in stop):
      s = sno.stem(w).encode('utf8')
      filtered_words.append(s)
      if (final_data['Score'].values)[i] == 'positive':
        positive_words.append(s)
      if (final_data['Score'].values)[i] == 'negative':
        negative_words.append(s)
      else:
        continue
    else:
      continue
  
  sqrl = b" ".join(filtered_words)
  filtered_sentence.append(sqrl)
  i += 1

len(filtered_sentence)

364171

In [0]:
final_data['cleaned_text'] = filtered_sentence

In [0]:
conn = sqlite3.connect('./final.sqlite')
c = conn.cursor()
conn.text_factory = str
final_data.to_sql('Reviews', conn, if_exists='replace')

In [0]:
con = sqlite3.connect('./final.sqlite')

In [0]:
data = pd.read_sql_query('''
select * from Reviews''', con)

In [35]:
data.shape

(364171, 12)

In [36]:
data.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,cleaned_text
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak book watch realli rosi movi...
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...
3,138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,b'great littl book read aloud- nice rhythm wel...
4,138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,b'book poetri month year goe month cute littl ...


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(ngram_range = (1,2))

In [0]:
final_tfidf = tfidf_vect.fit_transform(data['cleaned_text'].values)

In [41]:
final_tfidf.shape

(364171, 3058839)

In [42]:
features = tfidf_vect.get_feature_names()
len(features)

3058839

In [45]:
features[10000:10010]

['180mg hey',
 '180mg omega',
 '180mg polyphenol',
 '180mg potassium',
 '180mg sodium',
 '180mg total',
 '180ml',
 '180ml batch',
 '180ml caraf',
 '180ml hot']